In [ ]:
import cv2
import face_recognition
from tkinter import Tk, Label, Button
from PIL import Image, ImageTk


class FaceDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Live Face Detection")

        # Initialize the webcam
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("Error: Cannot access webcam.")
            return

        # Set up the video feed display in Tkinter
        self.label = Label(root)
        self.label.pack()

        # Add a quit button
        self.quit_button = Button(root, text="Quit", command=self.quit_app)
        self.quit_button.pack()

        # Start updating the video feed
        self.update_video_feed()

    def update_video_feed(self):
        # Read a frame from the webcam
        ret, frame = self.cap.read()
        if not ret:
            print("Error: Cannot read frame from webcam.")
            self.quit_app()
            return

        # Resize the frame for faster processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

        # Detect faces in the resized frame
        face_locations = face_recognition.face_locations(rgb_small_frame)

        # Print detected face locations only if faces are found
        if face_locations:
            print(f"Faces detected: {face_locations}")

        # Scale face locations back to original frame size
        face_locations = [(top * 4, right * 4, bottom * 4, left * 4) for (top, right, bottom, left) in face_locations]

        # Draw rectangles around detected faces
        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

        # Convert the frame to an image format suitable for Tkinter
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=img)

        # Update the label with the new image
        self.label.imgtk = imgtk
        self.label.configure(image=imgtk)

        # Schedule the next frame update
        self.root.after(10, self.update_video_feed)

    def quit_app(self):
        # Release the webcam and destroy the GUI
        if self.cap.isOpened():
            self.cap.release()
        self.root.destroy()


# Create the Tkinter application window
root = Tk()
app = FaceDetectionApp(root)
root.mainloop()